In [104]:
import netCDF4 as nc
import geopandas as gpd
import pandas as pd
import xarray as xr
# import native python garbage collector to explicitly invoke it
import gc

In [ ]:
# raw data file is not saved in repo, and can be downloaded as a zip file from its source:
# http://globalchange.bnu.edu.cn/research/soilwd.jsp "Soil organic carbon density: SOCD5min.zip"
# read the data file in with xarray and assign it to ds variable
DS = xr.open_dataset("./data/SOCD5min.nc")

In [ ]:
# transform it to a dataframe assigned to df variable
df = DS.to_dataframe()

In [ ]:
df.index.names

In [ ]:
# try the subset to remove na values without going through the flattening of the index first
# it works without resetting the index first
# flatten the index of the overall dataframe (after NaN removed, before summary) in place so it persists
# DO NOT DROP THE INDEX HERE! Flatten the multi-level index, and keep all levels as columns
df1 = df[~df.SOCD.isna()].reset_index()

In [ ]:
df1.head()

In [ ]:
df1.index.names

In [ ]:
# take only the 4.5 depth records and
# reset the index and drop the extra previous index column
df2 = df1[df1['depth'] == 4.5].reset_index(drop=True)

In [ ]:
df2.index.names

In [ ]:
df2.columns

In [ ]:
# for the dataframe with NA removed
# translate lon and lat columns into a spatial geometry variable in a GeoPandas dataframe, and
# set a CRS on the geo dataframe object according to data source documentation,
# readme file is available at http://globalchange.bnu.edu.cn/download/doc/worldsoil/readme.zip
gdf = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2['lon'], df2['lat']), crs="EPSG:4326")
# gdf.crs = "EPSG:4326" # moved above

In [ ]:
gdf.index.names

In [ ]:
gdf.lon

In [ ]:
gdf.columns

In [ ]:
# use the world dataset from geopandas to do a spatial link/merge with on points geometry which is a shapely series
# to grab each soil measurement location's country from
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.index.names

In [ ]:
world.columns

In [ ]:
# join my geometry from gdf2flatsurface to the world and
# get the countries they are residing in using a spatial join (sjoin)
gdf1 = gpd.sjoin(gdf, world, how='left').reset_index(drop=True)
# print(gdf.head())
#del world

In [ ]:
gdf1.index.names

In [ ]:
gdf1.columns

In [ ]:
# rename name column to country_name, in place to replace column name in same column, and
# rename other columns for clarity after they are merged with food trade data later
gdf1.rename(columns={
    "continent": "Reporter_Country_continent",
    "name": "Reporter_Country_name",
    "pop_est": "Reporter_Country_pop_est",
    "iso_a3": "Reporter_Country_ISO3",
    "gdp_md_est": "Reporter_Country_gdp_md_est",
    "lon": "Reporter_Country_lon",
    "lat": "Reporter_Country_lat",
    "SOCD": "Reporter_Country_SOCD_depth4_5"
}, inplace=True)

In [ ]:
# drop NaN i.e. null values from the result of linking gdf to add country from world dataset
# plotting those locations showed they are off land in water areas
# and drop the extra index column
gdf2 = gdf1.dropna().reset_index(drop=True)
# print(gdf.head())

In [ ]:
gdf2['Reporter_Country_lon'].count()

In [ ]:
# drop columns I no longer need from soil data to limit size
gdf3 = gdf2.drop(['geometry', 'index_right', 'depth'], axis=1)

In [ ]:
gdf3.head()

In [ ]:
gdf3.index.names

In [ ]:
#try converting geopandas dataframe back to standard pandas dataframe before more processing
df3 = pd.DataFrame(gdf3)
# check variable object type that should now be a <class 'pandas.core.frame.DataFrame'>
type(df3)
# then view the data types of the column series to see if any of them are still holding onto geopandas.array type
df3.dtypes
# del gdf

In [ ]:
df3.head()

In [ ]:
df3[df3['Reporter_Country_SOCD_depth4_5'].isna()]

In [ ]:
# get a unique list of the stats from the soil data for countries, to append to food data for food chart
# whilst converting geopandas dataframe back to standard pandas dataframe before more processing
# removed 'SOCDcountryMean' because of errors noted above
CntryStats = pd.DataFrame(df3, columns=[
    'Reporter_Country_continent', 'Reporter_Country_name',
    'Reporter_Country_ISO3', 'Reporter_Country_gdp_md_est',
    'Reporter_Country_pop_est'
    ]).drop_duplicates()

In [ ]:
CntryStats.head()

In [ ]:
# write intermittent full on land soil dataset of 4.5 cm depth to file for peripheral charts, e.g. bar chart
df3.to_csv("./data/dfsoil.csv")

In [ ]:
# load in the food trade detailed matrix copy freshly downloaded
# from https://www.fao.org/faostat/en/#data/TM to an alternate directory
# adding , encoding = "ISO-8859-1" to resolve "UnicodeDecodeError:
# 'utf-8' codec can't decode byte 0xf4 in position 38698: invalid continuation byte"
# alternately use the alias 'latin' for encoding
tr = '/Users/kathrynhurchla/Documents/hack_mylfs_GitHub_projects/Trade_DetailedTradeMatrix_E_All_Data_(Normalized).csv'
dffood = pd.read_csv(tr, encoding="ISO-8859-1")
# del tr

In [ ]:
# filter for just the 'Export  Quantity' rows by its element code identified earlier
dffood1 = dffood[dffood['Element Code'] == 5910].reset_index(drop=True)

In [ ]:
# rename Value to explicitly specify the type of value, 
# because I will drop Element column which is now only 'Export Quantity'
# and also drop Unit which is now all 'tonnes' (and will be labelled in plots clearly)
dffood1.rename(columns={"Value": "Export_Quantity_2019_Value_tonnes",
                       "Reporter Country Code": "Reporter_Country_Code"}, inplace=True)

In [ ]:
dffood1.columns

In [ ]:
# for my web app I will remove the export quantity trade rows where year is not 2019,
# i.e. I will keep only the most recent export dataset available
# naming it to a new dataframe whilst resetting index and dropping the previous index
dffood2 = dffood1.drop(dffood1.loc[dffood1['Year'] != 2019].index,
                     inplace=False).reset_index(drop=True)

In [ ]:
dffood2.head(3)

In [ ]:
# drop columns from food trade data that I do not need anymore
dffood3 = dffood2.drop(['Element Code',
                        'Element',
                        'Year Code',
                        'Unit',
                        'Item Code'], axis=1
                       )

In [ ]:
dffood3.columns

In [ ]:
dffood3.head(3)

In [ ]:
# read in the FAOSTAT key dataset as a variable (reusing ds)
DS1 = pd.read_csv('/Users/kathrynhurchla/Documents/hack_mylfs_GitHub_projects/FAOSTAT_data_11-26-2021.csv')
# rename the country code column to match exactly for merging easier
DS1.rename(columns={"Country Code": "Reporter_Country_Code", "ISO3 Code": "Reporter_Country_ISO3"}, inplace=True)

In [ ]:
DS1.index.names

In [ ]:
DS1.columns

In [ ]:
DS1.Reporter_Country_ISO3

In [ ]:
# for my web app I will remove the export quantity trade rows where year is not 2019,
# i.e. I will keep only the most recent export dataset available
# naming it to a new dataframe whilst resetting index and dropping the previous index
dffood4 = dffood3.drop(dffood3.loc[dffood3['Year'] != 2019].index,
                       inplace=False).reset_index(drop=True)

In [ ]:
dffood4.head(3)

In [ ]:
dffood5 = dffood3.drop(['Year'], axis=1
                       )

In [ ]:
# For dffood:
# using pandas merge function, link the trade matrix **reporter country** code with key to append its ISO_3 code,
# not necessary to do this for the partner country because I do not intend to connect that with food trade data
# with left data as food trade matrix
dffood6 = dffood5.merge(DS1[['Reporter_Country_Code', 'Reporter_Country_ISO3']],
                        # left on key column that exists with same name now in both dataframes
                        how='left',
                        on='Reporter_Country_Code',
                        )

In [ ]:
dffood6.head(3)

In [ ]:
# rename a few more columns to align across datasets for clarity
dffood6.rename(columns={"Reporter Countries": "Reporter_Country_name",
                        "Partner Countries": "Partner_Country_name"}, inplace=True
               )

In [ ]:
dffood6.columns

In [ ]:
# There is no "China" in the food trade matrix. I will fill China, mainland with China's ISO3 code 'CHN'
# find rows with the Reporter Country Code 41 (for China, mainland), 
# locate the 'ISO3 Code' column in those rows and set it to 'CHN'
dffood6.loc[dffood6['Reporter_Country_Code'] == 41, 'Reporter_Country_ISO3'] = 'CHN'

In [ ]:
# limit for scale to US partners in food dataset
USTradePartners = dffood6[dffood6['Partner Country Code'].isin([231, 232, 240])] 
                    
USTradePartners

In [ ]:
USTradePartners1 = USTradePartners['Reporter_Country_ISO3'].unique()
USTradePartners1

In [ ]:
(9*18)-5

In [231]:
# take subset of soil dataframe for only those reporter countries exporting food items to US
df_subUS = df3.loc[df3['Reporter_Country_ISO3'].isin(USTradePartners1)]

In [ ]:
df_subUS.head(3)

In [ ]:
# try to expand scale to both USA and China partners in food dataset
USCNTradePartners = dffood6[dffood6['Partner Country Code'].isin([41, 96, 128, 214, 351])]
USCNTradePartners

In [ ]:
USCNTradePartners1 = USCNTradePartners['Reporter_Country_ISO3'].unique()
USCNTradePartners1

In [ ]:
(9*18)-6

In [240]:
# take subset of soil dataframe for only those reporter countries exporting food items to US or China
df_subUSCN = df3.loc[df3['Reporter_Country_ISO3'].isin(USCNTradePartners1)]

In [ ]:
df_subUSCN

In [242]:
# write dataframes to file in case the kernel chokes on memory again from this point on
df_subUS.to_csv('./data/dfsoil_subUS.csv')
df_subUSCN.to_csv('./data/dfsoil_subUSCN.csv')

In [ ]:
# drop columns I do not need anymore
dffood7 = dffood6.drop(['Reporter_Country_Code', 'Partner Country Code'], axis=1)

In [245]:
# with left data as full 2019 food trade matrix, merge in unique country stats, for peripheral charts of at risk foods
dffood8 = dffood7.merge(CntryStats,
                        # left on key that exists with same name now in both dataframes
                        how='left',
                        on='Reporter_Country_ISO3'
                        )

In [ ]:
# write dataframe to file in case the kernel chokes on memory again from this point on
dffood8.to_csv('./data/dffood.csv')

In [ ]:
dffood8.columns

In [ ]:
# view the list of variable objects in memory with
[dir()]

In [ ]:
# selectively delete things I saved that I no longer need from within this script.
# Keep all '__...__' globals and aliases for packages unless I am done with them
# (it runs periodically otherwise to release unreferenced memory)
# review list of tracked object (generation=None) (generation=0) (generation=1) (generation=2)
gc.get_objects(generation=None)

In [254]:
del [CntryStats,
     DS,
     DS1,
     USCNTradePartners,
     USCNTradePartners1,
     USTradePartners,
     USTradePartners1,
     df,
     df1,
     df2,
     df3,
     dffood,
     dffood1,
     dffood2,
     dffood3,
     dffood4,
     dffood5,
     dffood6,
     dffood7,
     gdf,
     gdf1,
     gdf2,
     gdf3,
     gpd,
     nc,
     tr,
     world,
     xr
    ]
     
     

In [ ]:
# release selected memory
gc.collect

In [ ]:
# view the list of variable objects in memory with
[dir()]

In [257]:
%store df_subUS

Stored 'df_subUS' (DataFrame)


In [258]:
%store df_subUSCN

Stored 'df_subUSCN' (DataFrame)


In [259]:
%store dffood8

Stored 'dffood8' (DataFrame)


In [1]:
df_subUSCN.head()

NameError: name 'df_subUSCN' is not defined

In [ ]:
# # pandas merge US food dataset with soil filtered for only US partners
# df_subUS1 = dffood8.merge(df_subUS,
#                          how='inner',
#                          on='Reporter_Country_ISO3'
#                          )

In [ ]:
# if merge works, print a concise summary of the resulting dataframe
print(df_subUS1.info)

In [ ]:
# write a CSV of merged US partner 4.5 depth socd merged with food trade data to the data folder in project directory
df_subUS1.to_csv("./data/df_subUS1.csv")

In [ ]:
# del df_subUS
# del df_subUS1

In [ ]:
# # pandas merge North American food dataset with soil filtered for only NA partners
# df_subUSCN1 = dffood8.merge(df_subUSCN,
#                             how='inner',
#                             on='Reporter_Country_ISO3'
#                             )

In [ ]:
# if merge works, print a concise summary of the resulting dataframe
print(df_subUSCN1.info)

In [ ]:
# # write a CSV of merged NA partner 4.5 depth socd merged with food trade data to the data folder in project directory
# df_subUSCN1.to_csv("./data/df_subUSCN1.csv")